## Accessing reanalysis data with xarray

### imports 

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['font.size'] = 14
plt.rcParams['lines.linewidth'] = 2

In [ ]:
from tqdm.notebook import tqdm_notebook

In [ ]:
import io
import requests 
import pathlib

In [ ]:
import xarray as xr
from cartopy import crs as ccrs
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
# cluster = LocalCluster(
#     n_workers=6,
#     threads_per_worker=1,
#     memory_limit="4GiB",
#     processes=True,
# )

# client = Client(cluster)

# client 

### Monthly NCEP / NCAR reanalysis, via HTTPS 

start here: https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.derived/ 

In [ ]:
url = 'https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.derived/surface/air.mon.mean.nc'

In [ ]:
# xr.open_dataset(url)

In [ ]:
with requests.get(url) as r:
    dset = xr.open_dataset(io.BytesIO(r.content))

In [ ]:
dset

In [ ]:
dset.close()

### OpenDAP protocol, served by the PSL Thredds server 

see catalog: [https://psl.noaa.gov/thredds/catalog/Datasets/catalog.html](https://psl.noaa.gov/thredds/catalog/Datasets/catalog.html)

In [ ]:
url = 'https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/surface/air.mon.mean.nc'

In [ ]:
dset = xr.open_dataset(url)

In [ ]:
dset

In [ ]:
dset.close()

### dataset split between multiple files: daily reanalyses via OpenDAP

see: [https://psl.noaa.gov/thredds/catalog/Datasets/ncep.reanalysis.dailyavgs/surface/catalog.html](https://psl.noaa.gov/thredds/catalog/Datasets/ncep.reanalysis.dailyavgs/surface/catalog.html)

each file has the pattern: 

`https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.YYYY.nc`

In [ ]:
dset = xr.open_dataset('https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.2000.nc')

In [ ]:
dset

In [ ]:
dset.close()

#### create a list of URLs 

In [ ]:
base_url = 'https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995'

In [ ]:
list_files = [f'{base_url}.{year}.nc' for year in range(2010, 2021)]

In [ ]:
list_files

### open a *multiple files* dataset with `open_mfdataset`

In [ ]:
dset = xr.open_mfdataset(list_files, parallel=True)

In [ ]:
dset = dset.drop('time_bnds')

In [ ]:
dset

In [ ]:
dset['air'].isel(time=0).plot()

## conversion from Kelvin to Celsius is handled easily 

In [ ]:
dset['air'] = dset['air'] - 273.15

# or 

# dset['air'] -= 273.15


In [ ]:
dset['air'].isel(time=0).plot()

In [ ]:
type(dset.air.data)

### how is the data actually represented ?

In [ ]:
dset.chunks

### to calculate the time mean, it is wise to rechunk

In [ ]:
dset = dset.chunk({'time':-1, 'lat':10, 'lon':10})

In [ ]:
dset.chunks

In [ ]:
dset_ave = dset.mean('time')

In [ ]:
with ProgressBar(): 
    dset_ave.compute() 

In [ ]:
dset_ave['air'].plot()

In [ ]:
dset_ave.close()
dset.close()

### When opening a Multiple File Dataset with `xr.open_mfdataset` you can specify a preprocessing function (e.g. extraction of a geographical domain)

In [ ]:
def preprocess(dset): 
    dset = dset.sortby('lat')
    dset = dset.sel(lat=slice(-55, -30), lon=slice(140, 190))
    return dset

In [ ]:
dset = xr.open_mfdataset(list_files, parallel=True, preprocess=preprocess)

In [ ]:
dset.isel(time=0)['air'].plot()

In [ ]:
dset.close()

-----

### ERA5 via OpenDAP on the Asia Pacific Data Research Center 

NOTE: the full ERA5 reanalysis data (all variables, all levels, etc) is available at [https://cds.climate.copernicus.eu/#!/home](https://cds.climate.copernicus.eu/#!/home)

start at http://apdrc.soest.hawaii.edu/data/data.php

### monthly aggregated variables 

In [ ]:
url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/monthly_2d/Surface'

In [ ]:
dset = xr.open_dataset(url)

In [ ]:
dset

### we can just extract the variable of interest 

In [ ]:
dset = xr.open_dataset(url)[['t2m']]

In [ ]:
dset.nbytes / 1e9

In [ ]:
dset

In [ ]:
dset.close()

### daily aggregated variables 

- [http://apdrc.soest.hawaii.edu/dods/public_data/Reanalysis_Data/ERA5/daily_3d](http://apdrc.soest.hawaii.edu/dods/public_data/Reanalysis_Data/ERA5/daily_3d)

air temperature: 

- [http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/daily_3d/Air_Temperature](http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/daily_3d/Air_Temperature)



In [ ]:
url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/daily_3d/Air_Temperature'

In [ ]:
dset = xr.open_dataset(url)

In [ ]:
dset

In [ ]:
dset.nbytes / 1e9

In [ ]:
dset.lev

In [ ]:
dset = dset.sel(lev=1000., lat=slice(-90., 30.))

In [ ]:
dset.nbytes / 1e9

### we are going to save the 1st 5 days worth of data to disk 

In [ ]:
opath = pathlib.Path('../data/ERA5/temperature')

In [ ]:
if not(opath.exists()): 
    print(f"{str(opath)} does not exist, creating ...")
    opath.mkdir(parents=True)

In [ ]:
for t in tqdm_notebook(dset.time.to_index()[:5]): 
    sub = dset.sel(time=t)
    sub.expand_dims({'time':[t]})
    sub.to_netcdf(opath.joinpath(f'./ERA5_temp_{t:%Y-%m-%d}.nc'))
    sub.close()

In [ ]:
dset.close()

### list the local files 

In [ ]:
lfiles = list(opath.glob("ERA5_temp_*.nc"))

In [ ]:
lfiles.sort()

In [ ]:
lfiles

In [ ]:
dset = xr.open_mfdataset(lfiles, concat_dim='time', combine='nested')

In [ ]:
dset

### convert to Celsius 

In [ ]:
dset['temperature'] -= 273.15

### re-chunk, we want one chunk along the time dimension only as we are going to calculate the time mean 

In [ ]:
dset.chunks

In [ ]:
dset = dset.chunk({'time':-1, 'lat':10, 'lon':10})

In [ ]:
dset.chunks

In [ ]:
dset = dset.mean('time')

In [ ]:
dset.compute()

### proper map with coastlines, using cartopy 

In [ ]:
cbar_kwargs = dict(shrink=0.5, orientation='horizontal', pad=0.01)

In [ ]:
f, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree(central_longitude=180)})
dset['temperature'].plot(ax=ax, levels=20, transform=ccrs.PlateCarree(), cbar_kwargs=cbar_kwargs)
ax.coastlines()
ax.set_extent([0, 360, -90, 20], crs=ccrs.PlateCarree())
ax.set_title('ERA5 temperature at 1000 hPa')

In [ ]:
cbar_kwargs.update(dict(orientation='vertical', shrink=0.7))

In [ ]:
f, ax = plt.subplots(subplot_kw={"projection":ccrs.Orthographic(central_longitude=150, central_latitude=-55)})
dset['temperature'].plot(ax=ax, levels=20, transform=ccrs.PlateCarree(), cbar_kwargs=cbar_kwargs)
ax.coastlines(resolution='10m')
ax.set_title('ERA5 temperature at 1000 hPa')

In [ ]:
dset.close()